# Work in progress

# Using deeplearning to predict Fantasy Premier League players performance 

Specifically, four categories of players have been identified using K-means clustering (see FPL data exploration notebook for now):

(1) Low cost, low points <br>
(2) Low cost, high points <br>
(3) High cost, low points <br>
(4) High cost, high points

Tensorflow will be used to see if it is possible to predict which category a player will be in for the current season, based on the player data for the previous season.


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns


Bad key "text.kerning_factor" on line 4 in
C:\Users\User\anaconda3\envs\py3-TF2.0\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Preprocess dataset for Tensorflow

In [2]:
data = pd.read_csv('player_categories.csv')
data

,name,assists,bonus,bps,clean_sheets,creativity,end_cost,goals_conceded,goals_scored,ict_index,...,saves,start_cost,threat,total_points,yellow_cards,cost_18_19,cat_1,cat_2,cat_3,cat_4
0,Aaron_Cresswell_376,7,10,613,10,562.7,49,60,1,145.0,...,0,50,193.0,118,7,55,0,0,0,1
1,Aaron_Lennon_430,4,0,204,6,203.2,54,30,0,73.3,...,0,55,297.0,65,2,50,0,0,0,1
2,Aaron_Mooy_516,3,8,576,10,848.8,52,52,4,195.2,...,0,55,350.0,113,4,55,0,0,0,1
3,Aaron_Wan-Bissaka_122,0,0,116,2,60.0,45,10,0,26.8,...,0,45,2.0,15,1,40,0,1,0,0
4,Abdoulaye_Doucouré_368,4,16,679,9,595.8,52,63,7,199.7,...,0,50,537.0,136,10,60,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Wilfred_Ndidi_176,4,4,449,9,230.7,49,45,0,117.2,...,0,50,299.0,78,6,50,0,0,0,1
211,Wilfried_Zaha_133,7,8,404,10,716.9,71,31,9,251.0,...,0,70,1125.0,136,5,70,0,1,0,0
212,Willian_Borges Da Silva_113,8,13,522,9,891.5,70,19,6,224.5,...,0,70,715.0,130,1,75,0,0,1,0
213,Will_Hughes_370,3,5,190,4,267.0,46,16,2,64.3,...,0,50,136.0,53,0,50,0,0,0,1


In [3]:
inputs = data[['assists', 'bonus', 'bps', 'clean_sheets', 'end_cost',
       'goals_conceded', 'goals_scored', 'minutes',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'saves', 'start_cost', 'total_points',
       'yellow_cards', 'cost_18_19', 'ict_index']]
targets = data[['cat_1', 'cat_2', 'cat_3', 'cat_4']]

scaler = StandardScaler()
scaler.fit(inputs)

inputs_scaled_all = scaler.transform(inputs)

targets_all = targets.to_numpy()

shuffled_indices = np.arange(inputs_scaled_all.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = inputs_scaled_all[shuffled_indices]
shuffled_targets = targets_all[shuffled_indices]

In [4]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(train_samples_count)
print(validation_samples_count)
print(test_samples_count)

172
21
22


In [5]:
np.savez('player_cat_data_train', inputs=train_inputs, targets=train_targets)
np.savez('player_cat_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('player_cat_data_test', inputs=test_inputs, targets=test_targets)

## Data - haven't balanced

In [6]:
# create temporary variable npz, stores the three datasets
npz = np.load('player_cat_data_train.npz')

# extract inputs using keywords
# ensure all floats
train_inputs, train_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)

# load validation data in temporary variable
npz = np.load('player_cat_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# load test data in the temporary variable
npz = np.load('player_cat_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Model

In [7]:
# set parameters
input_size = train_inputs.shape[1]
output_size = train_targets.shape[1]
hidden_layer_size = 50
    
# define model
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3rd hidden layer
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 4th hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
# sparse for label vectors e.g. 0,1,2; not one hot encoded targets
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )

Train on 172 samples, validate on 21 samples
Epoch 1/100
172/172 - 2s - loss: 1.6373 - accuracy: 0.1163 - val_loss: 1.7489 - val_accuracy: 0.0476
Epoch 2/100
172/172 - 0s - loss: 1.5469 - accuracy: 0.1453 - val_loss: 1.6560 - val_accuracy: 0.0476
Epoch 3/100
172/172 - 0s - loss: 1.4676 - accuracy: 0.1744 - val_loss: 1.5696 - val_accuracy: 0.0952
Epoch 4/100
172/172 - 0s - loss: 1.3920 - accuracy: 0.2442 - val_loss: 1.4927 - val_accuracy: 0.0952
Epoch 5/100
172/172 - 0s - loss: 1.3332 - accuracy: 0.3314 - val_loss: 1.4207 - val_accuracy: 0.1905
Epoch 6/100
172/172 - 0s - loss: 1.2738 - accuracy: 0.4477 - val_loss: 1.3549 - val_accuracy: 0.3810
Epoch 7/100
172/172 - 0s - loss: 1.2196 - accuracy: 0.5640 - val_loss: 1.2942 - val_accuracy: 0.4286
Epoch 8/100
172/172 - 0s - loss: 1.1691 - accuracy: 0.6453 - val_loss: 1.2380 - val_accuracy: 0.6667
Epoch 9/100
172/172 - 0s - loss: 1.1230 - accuracy: 0.6919 - val_loss: 1.1847 - val_accuracy: 0.6667
Epoch 10/100
172/172 - 0s - loss: 1.0790 - acc

## Analyse model

In [8]:
y_hat = pd.DataFrame(np.squeeze(model.predict_on_batch(train_inputs)).round().astype(int)) # problem, can't index
results = pd.concat([y_hat, pd.DataFrame(train_targets)], axis=1)
results.columns = [['prob_1', 'prob_2', 'prob_3', 'prob_4', 'cat_1', 'cat_2', 'cat_3', 'cat_4']]
results

,prob_1,prob_2,prob_3,prob_4,cat_1,cat_2,cat_3,cat_4
0,0,0,1,0,0,0,1,0
1,0,0,0,1,0,0,0,1
2,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,1
4,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
167,0,0,0,1,0,0,0,1
168,1,0,0,0,1,0,0,0
169,0,0,0,1,0,0,0,1
170,0,1,0,0,0,1,0,0


In [9]:
print(np.sum(results.iloc[:,0] == results.iloc[:,4])/len(results), 
      np.sum(results.iloc[:,1] == results.iloc[:,5])/len(results), 
      np.sum(results.iloc[:,2] == results.iloc[:,6])/len(results), 
      np.sum(results.iloc[:,3] == results.iloc[:,7])/len(results)
     )


0.9941860465116279 0.8313953488372093 0.9651162790697675 0.8604651162790697


In [10]:
results.iloc[:,1] = results.iloc[:,1].map({1:2, 0:0})
results.iloc[:,2] = results.iloc[:,2].map({1:3, 0:0})
results.iloc[:,3] = results.iloc[:,3].map({1:4, 0:0})
results

,prob_1,prob_2,prob_3,prob_4,cat_1,cat_2,cat_3,cat_4
0,0,0,3,0,0,0,1,0
1,0,0,0,4,0,0,0,1
2,0,0,0,0,0,1,0,0
3,0,0,0,4,0,0,0,1
4,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
167,0,0,0,4,0,0,0,1
168,1,0,0,0,1,0,0,0
169,0,0,0,4,0,0,0,1
170,0,2,0,0,0,1,0,0


In [11]:
results['categories'] = results.iloc[:,0:4].idxmax(axis=1)
results

,prob_1,prob_2,prob_3,prob_4,cat_1,cat_2,cat_3,cat_4,categories
0,0,0,3,0,0,0,1,0,"(prob_3,)"
1,0,0,0,4,0,0,0,1,"(prob_4,)"
2,0,0,0,0,0,1,0,0,"(prob_1,)"
3,0,0,0,4,0,0,0,1,"(prob_4,)"
4,0,0,0,0,0,0,1,0,"(prob_1,)"
...,...,...,...,...,...,...,...,...,...
167,0,0,0,4,0,0,0,1,"(prob_4,)"
168,1,0,0,0,1,0,0,0,"(prob_1,)"
169,0,0,0,4,0,0,0,1,"(prob_4,)"
170,0,2,0,0,0,1,0,0,"(prob_2,)"


In [12]:
results.iloc[:,8] = results.iloc[:,8].map({'(prob_1,)':1, '(prob_2,)':2, '(prob_3,)':3, '(prob_4,)':4,})
results

,prob_1,prob_2,prob_3,prob_4,cat_1,cat_2,cat_3,cat_4,categories
0,0,0,3,0,0,0,1,0,NaN
1,0,0,0,4,0,0,0,1,NaN
2,0,0,0,0,0,1,0,0,NaN
3,0,0,0,4,0,0,0,1,NaN
4,0,0,0,0,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...
167,0,0,0,4,0,0,0,1,NaN
168,1,0,0,0,1,0,0,0,NaN
169,0,0,0,4,0,0,0,1,NaN
170,0,2,0,0,0,1,0,0,NaN


In [53]:
#TypeError: only integer scalar arrays can be converted to a scalar index
#results['prob_1'] == results['cat_1']

TypeError: only integer scalar arrays can be converted to a scalar index

## Test model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))